In [2]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os
import time
import pyttsx3
import playsound
import speech_recognition as sr
from gtts import gTTS

SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

In [3]:
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = r.listen(source)
        said = ""

    try:
        print("Recognizing...")
        said = r.recognize_google(audio)
        print(f"You said: {said}")
    except Exception as e:
        print("Exception: " + str(e))

    return said


In [6]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.Collecting google-api-python-client
     ---------------------------------------- 12.8/12.8 MB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 96.9/96.9 kB 5.4 MB/s eta 0:00:00
     -------------------------------------- 122.2/122.2 kB 7.5 MB/s eta 0:00:00
     ------------------------------------- 230.9/230.9 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.0.0
    Uninstalling google-auth-oauthlib-1.0.0:
      Successfully uninstalled google-auth-oauthlib-1.0.0



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.13.0 requires google-auth-oauthlib<1.1,>=0.5, but you have google-auth-oauthlib 1.1.0 which is incompatible.


In [ ]:
pip install pyttsx3

In [4]:
import pytz
import pyttsx3
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

MONTHS =["january","february","march","april","may","june","july","august","september","october","november","december"]
DAYS=["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
DAY_EXTENTIONS = ["rd","th","st"]

def authenticate_google():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES)
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  service = build("calendar", "v3", credentials=creds)
    
  return service
def get_events(date,service):
    # Call the Calendar API
    date = datetime.datetime.combine(date,datetime.datetime.min.time())
    end_date = datetime.datetime.combine(date,datetime.datetime.max.time())
    utc = pytz.UTC
    date = date.astimezone(utc)
    end_date = end_date.astimezone(utc)
    
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=date.isoformat() ,timeMax=end_date.isoformat(),
            singleEvents=True,
            orderBy="startTime",).execute()
    )
    events = events_result.get("items", [])

    if not events:
      speak("No upcoming events found.")
    else:
      speak(f"You have {len(events)} events on this day")
      
      for event in events:
          start = event["start"].get("dateTime", event["start"].get("date"))
          print(start, event["summary"])
          start_time = str(start.split("T")[1].split("-")[0])
          if int(start_time.split(":")[0]) < 12:
            start_time = start_time + "am"
          else:
            start_time = str(int(start_time.split(":")[0])-12)
            start_time = start_time + "pm"
            
          speak(event["summary"]+ "at" + start_time)

def get_date(text):
    text = text.lower()
    today = datetime.date.today()  
    
    if text.count == "today" > 0:
        return today
    
    day = -1
    day_of_week = -1
    month = -1
    year = today.year
    
    for word in text.split():
        if word in MONTHS:
             month = MONTHS.index(word) + 1
        elif word in DAYS:
            day_of_week = DAYS.index(word)
        elif word.isdigit():
            day = int(word)
        else:
            for ext in DAY_EXTENTIONS:
                found = word.find(ext)
                if found>0:
                    try:
                        day = int(word[:found])
                    except:
                        pass

    if month<today.month and month != -1:
        year = year+1
        
    if day < today.day and month == -1 and day != -1:
        month = month+1
        
    if month == -1 and day == -1 and day_of_week != -1:
        current_day_of_week = today.weekday()
        dif = day_of_week = current_day_of_week
        
        if dif <0:
            dif +=7
            if text.count("next") >=1:
                dif += 7
        return today + datetime.timedelta(dif)
    if month == -1 or day == -1:
        return None
    return datetime.date(month=month,day=day,year=year)


SERVICE = authenticate_google()
text = get_audio()

CALENDER_STRS = ["What do i have","do i have plans","am i busy"]
for phrase in CALENDER_STRS:
    if phrase in text:
        date = get_date(text)
        if date:
            get_events(date, SERVICE)
        else:
            speak("Please Try Again")

get_events(get_date(text),SERVICE)

Listening...
Recognizing...
You said: what do I have in December 5th
2023-12-05T11:45:00+05:30 birthday


In [41]:
pip install pytz

Note: you may need to restart the kernel to use updated packages.
